<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TPUs_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

#Data process

In [ ]:
From 276c1138f2f97749b1156edbac47c70678611f0a Mon Sep 17 00:00:00 2001
From: zhengjun10 <zhengjun10@huawei.com>
Date: Wed, 19 May 2021 17:23:23 +0800
Subject: [PATCH] fix train example code

---
 build.sh                                                  | 12 ++++++------
 .../src/main/java/com/mindspore/lite/TrainSession.java    |  6 ------
 mindspore/lite/java/native/runtime/train_session.cpp      | 15 +--------------
 3 files changed, 7 insertions(+), 26 deletions(-)

diff --git a/build.sh b/build.sh
index cf8b8c2..a084c29 100755
--- a/build.sh
+++ b/build.sh
@@ -723,8 +723,8 @@ build_lite_java_arm64() {
     mkdir -p ${JAVA_PATH}/java/app/libs/arm64-v8a/
     mkdir -p ${JAVA_PATH}/native/libs/arm64-v8a/
     if [[ "X$SUPPORT_TRAIN" = "Xon" ]]; then
-      cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmindspore-lite.so ${JAVA_PATH}/java/app/libs/arm64-v8a/
-      cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmindspore-lite.so ${JAVA_PATH}/native/libs/arm64-v8a/
+      cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmindspore-lite-train.so ${JAVA_PATH}/java/app/libs/arm64-v8a/
+      cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmindspore-lite-train.so ${JAVA_PATH}/native/libs/arm64-v8a/
       cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmslite_kernel_reg.so ${JAVA_PATH}/java/app/libs/arm64-v8a/
       cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmslite_kernel_reg.so ${JAVA_PATH}/native/libs/arm64-v8a/
 
@@ -759,8 +759,8 @@ build_lite_java_arm32() {
     mkdir -p ${JAVA_PATH}/java/app/libs/armeabi-v7a/
     mkdir -p ${JAVA_PATH}/native/libs/armeabi-v7a/
     if [[ "X$SUPPORT_TRAIN" = "Xon" ]]; then
-      cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmindspore-lite.so ${JAVA_PATH}/java/app/libs/armeabi-v7a/
-      cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmindspore-lite.so ${JAVA_PATH}/native/libs/armeabi-v7a/
+      cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmindspore-lite-train.so ${JAVA_PATH}/java/app/libs/armeabi-v7a/
+      cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmindspore-lite-train.so ${JAVA_PATH}/native/libs/armeabi-v7a/
       cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmslite_kernel_reg.so ${JAVA_PATH}/java/app/libs/armeabi-v7a/
       cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmslite_kernel_reg.so ${JAVA_PATH}/native/libs/armeabi-v7a/
 
@@ -796,8 +796,8 @@ build_lite_java_x86() {
     mkdir -p ${JAVA_PATH}/java/linux_x86/libs/
     mkdir -p ${JAVA_PATH}/native/libs/linux_x86/
     if [[ "X$SUPPORT_TRAIN" = "Xon" ]]; then
-      cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmindspore-lite.so ${JAVA_PATH}/java/linux_x86/libs/
-      cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmindspore-lite.so ${JAVA_PATH}/native/libs/linux_x86/
+      cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmindspore-lite-train.so ${JAVA_PATH}/java/linux_x86/libs/
+      cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmindspore-lite-train.so ${JAVA_PATH}/native/libs/linux_x86/
       cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmslite_kernel_reg.so ${JAVA_PATH}/java/linux_x86/libs/
       cp ${BASEPATH}/mindspore/lite/build/java/${JTARBALL}/train/lib/libmslite_kernel_reg.so ${JAVA_PATH}/native/libs/linux_x86/
 
diff --git a/mindspore/lite/java/java/common/src/main/java/com/mindspore/lite/TrainSession.java b/mindspore/lite/java/java/common/src/main/java/com/mindspore/lite/TrainSession.java
index 3692510..b056bd1 100644
--- a/mindspore/lite/java/java/common/src/main/java/com/mindspore/lite/TrainSession.java
+++ b/mindspore/lite/java/java/common/src/main/java/com/mindspore/lite/TrainSession.java
@@ -149,10 +149,6 @@ public class TrainSession {
     public boolean setupVirtualBatch(int virtualBatchMultiplier) {
         return this.setupVirtualBatch(this.sessionPtr, virtualBatchMultiplier, -1.0f, -1.0f);
     }
-
-    public boolean setLossName(String lossName) {
-        return this.setLossName(this.sessionPtr, lossName);
-    }
     
     
     private native long createSession(String modelFilename, long msConfigPtr);
@@ -190,6 +186,4 @@ public class TrainSession {
     private native boolean setLearningRate(long sessionPtr, float learning_rate);
 
     private native boolean setupVirtualBatch(long sessionPtr, int virtualBatchMultiplier, float learningRate, float momentum);
-
-    private native boolean setLossName(long sessionPtr, String lossName);
 }
diff --git a/mindspore/lite/java/native/runtime/train_session.cpp b/mindspore/lite/java/native/runtime/train_session.cpp
index 14d515e..8b8a96c 100644
--- a/mindspore/lite/java/native/runtime/train_session.cpp
+++ b/mindspore/lite/java/native/runtime/train_session.cpp
@@ -239,7 +239,7 @@ extern "C" JNIEXPORT jboolean JNICALL Java_com_mindspore_lite_TrainSession_saveT
     return (jboolean) false;
   }
   auto *train_session_ptr = static_cast<mindspore::session::TrainSession *>(session_pointer);
-  auto ret = train_session_ptr->SaveToFile(JstringToChar(env, model_file_name));
+  auto ret = train_session_ptr->Export(JstringToChar(env, model_file_name));
   return (jboolean)(ret == 0);
 }
 
@@ -318,16 +318,3 @@ extern "C" JNIEXPORT jboolean JNICALL Java_com_mindspore_lite_TrainSession_setup
   auto ret = train_session_ptr->SetupVirtualBatch(virtualBatchMultiplier, learningRate, momentum);
   return (jboolean)(ret == mindspore::lite::RET_OK);
 }
-
-extern "C" JNIEXPORT jboolean JNICALL Java_com_mindspore_lite_TrainSession_setLossName(JNIEnv *env, jobject thiz,
-                                                                                       jlong session_ptr,
-                                                                                       jstring lossName) {
-  auto *session_pointer = reinterpret_cast<void *>(session_ptr);
-  if (session_pointer == nullptr) {
-    MS_LOGE("Session pointer from java is nullptr");
-    return (jboolean) false;
-  }
-  auto *train_session_ptr = static_cast<mindspore::session::TrainSession *>(session_pointer);
-  auto ret = train_session_ptr->SetLossName(JstringToChar(env, lossName));
-  return (jboolean)(ret == mindspore::lite::RET_OK);
-}
-- 
2.7.4

